In [1]:
import xlrd
import sklearn
from sklearn.neural_network import MLPClassifier
import numpy as np
import os
import scipy.stats
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import load_model
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras import backend as K
from keras import regularizers
import numpy as np
from collections import Counter
from sklearn.linear_model import LogisticRegression
import scipy.stats
import random
import tensorflow as tf
from collections import defaultdict
from scipy import stats
import matplotlib.pyplot as plt 
from keras import optimizers
import time

/anaconda3/envs/nlu/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# open the excel file
loc = ("Premier_League_Data.xlsx")
wb = xlrd.open_workbook(loc)

In [3]:
num_parameters = 22
num_teams = 20
num_stats_sheets = 7
num_total_sheets = 13

In [11]:
# create the sheets

stats_sheets = []
for i in range(0, num_stats_sheets):
    stats_sheets.append(wb.sheet_by_index(i))


TypeError: 'Sheet' object does not support indexing

In [5]:
stats_by_season = []
teams_per_season = []
promoted_per_season = []
relegated_per_season = []

In [14]:
# stats_this_year = {}
# teams_this_year = []
# promoted_this_year = []
# relegated_this_year = []
for m in range(0, num_stats_sheets):
    stats_this_year = {}
    teams_this_year = []
    promoted_this_year = []
    relegated_this_year = []
    for i in range(1, num_teams + 1):
        team = stats_sheets[m].cell_value(i, 0)
        print(team)
        teams_this_year.append(team)
        stats_this_year[team] = []
        for j in range(1, num_parameters + 1):
            stats_this_year[team].append(stats_sheets[m].cell_value(i, j))
            
    relegated_this_year.append(teams_this_year[num_teams - 3])
    relegated_this_year.append(teams_this_year[num_teams - 2])
    relegated_this_year.append(teams_this_year[num_teams - 1])

#     print(stats_she)
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 3, 0))
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 4, 0))
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 5, 0))

    stats_by_season.append(stats_this_year)
    teams_per_season.append(teams_this_year)
    print(promoted_this_year)
    promoted_per_season.append(promoted_this_year)
    relegated_per_season.append(relegated_this_year)
    # print(teams_this_year)
#print(teams_per_season[0])
print(promoted_per_season)

Manchester City
Manchester United
Tottenham Hotspur
Chelsea
Liverpool
Arsenal
Burnley
Everton
Leicester City
Newcastle United
AFC Bournemouth
Crystal Palace
Watford
West Ham United
Brighton and Hove Albion
Huddersfield Town
Swansea City
Southampton
Stoke City
West Bromwich Albion


IndexError: list index out of range

In [8]:
# get promoted teams from 2010/11 (no other info on this sheet, so has to be hard-coded in)

stats_1011_sheet = wb.sheet_by_index(13)
title = stats_1011_sheet.cell_value(0, 0)
promoted_1011 = []
for i in range(1, 4):
    team = stats_1011_sheet.cell_value(i, 0)
    promoted_1011.append(team.strip())
promoted_per_season.append(promoted_1011)

IndexError: list index out of range

In [ ]:
# create latest test data
fixtures_1819_sheet = wb.sheet_by_index(14)
fixtures_1819 = []
stats_1718 = stats_by_season[0] # 1718 stats

for i in range(1, 41):
    home_team = fixtures_1819_sheet.cell_value(i, 1)
    away_team = fixtures_1819_sheet.cell_value(i, 2)
    if home_team in stats_1718.keys():
        home_stats = stats_1718[home_team]
    else:
        promoted_index = promoted_per_season[0].index(home_team)
        print(promoted_index)
        promoted_team = promoted_per_season[1][promoted_index] # out of bounds error?
        print(promoted_per_season[1])
        print(promoted_team)
        home_stats = stats_1718[promoted_team]
    if away_team in stats_1718.keys():
        away_stats = stats_1718[away_team]
    else:
        print("looking for: ", away_team)
        print(promoted_per_season[1])
        print(promoted_per_season[0])
        promoted_index = promoted_per_season[0].index(away_team)
        print(promoted_index)
        promoted_team = promoted_per_season[1][promoted_index] # out of bounds error?
        print(promoted_team)
        away_stats = stats_1718[promoted_team]
    instance = []
    for k in range (0, len(home_stats)):
        if (away_stats[k] == 0):
            ratio = (home_stats[k] + 1) / (away_stats[k] + 1)
        else:
            ratio = home_stats[k] / away_stats[k]
#           ratio = home_stats[k] - away_stats[k]
        instance.append(ratio)
    fixtures_1819.append(instance)
    
print(fixtures_1819)


In [ ]:
score_sheets = []
for i in range(num_stats_sheets, num_total_sheets):
    score_sheets.append(wb.sheet_by_index(i))


In [ ]:
all_results = []
all_stats = []
all_outcomes = []
stats_pairs_by_season = []
outcomes_by_season = []

for i in range (0 , 6):
    results = {}
    wins = 0
    losses = 0
    draws = 0
    for j in range (1, num_teams * (num_teams - 1) + 1):
        home_team = score_sheets[i].cell_value(j, 1).strip()
        away_team = score_sheets[i].cell_value(j, 2).strip()
        result = score_sheets[i].cell_value(j, 5)
        
        if result == "W":
            wins +=1 
            results[(home_team, away_team)] = 1
        elif result == "L":
            losses+=1
            results[(home_team, away_team)] = -1
        else:
            draws+=1
            results[(home_team, away_team)] = 0
    all_results.append(results)
    stats = []
    outcomes = []
    counter = 0
    for key in results.keys():
        counter = counter + 1
        # each home-away pair for that season
        home_team = key[0].strip()
        away_team = key[1].strip()
        prev_season_stats = stats_by_season[i + 1]
        if home_team in prev_season_stats.keys():
            home_stats = prev_season_stats[home_team]
        else:
            promoted_index = promoted_per_season[i+1].index(home_team)
            promoted_team = promoted_per_season[i+2][promoted_index] # out of bounds error?
            home_stats = prev_season_stats[promoted_team]
        if away_team in prev_season_stats.keys():
            away_stats = prev_season_stats[away_team]
        else:
            promoted_index = promoted_per_season[i + 1].index(away_team)
            promoted_team = promoted_per_season[i + 2][promoted_index] # out of bounds error?
            away_stats = prev_season_stats[promoted_team]
        instance = []
        for k in range (0, len(home_stats)):
            if (away_stats[k] == 0):
                ratio = (home_stats[k] + 1) / (away_stats[k] + 1)
            else:
                ratio = home_stats[k] / away_stats[k]
#             ratio = home_stats[k] - away_stats[k]
            instance.append(ratio)
        outcome = results[(home_team, away_team)]
        if (outcome !=0):
            stats.append(instance)
            all_stats.append(instance)
            outcomes.append(outcome)
            all_outcomes.append(outcome)
    stats_pairs_by_season.append(stats)
    outcomes_by_season.append(outcomes)
    print(wins, losses, draws, draws/(wins+losses+draws))
print(len(all_stats))
print(len(all_outcomes))
print(all_outcomes)
              

In [ ]:
num_classes = 2
x_train, x_test, y_train, y_test = train_test_split(all_stats, all_outcomes, test_size = 0.3)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

num_records = len(x_train)
num_features = len(x_train[0])


print("Training Feature matrix being passed into neural network has " + str(num_records) + 
      " vectors and " + str(num_features) + " parameters. ")

In [ ]:
def fit_maxent_classifier(X, y, testFeatureVectors): 
    mod = LogisticRegression(fit_intercept=True, multi_class = 'multinomial', solver = 'newton-cg')
    mod.fit(X, y)
    pred = mod.predict(testFeatureVectors)
    # print(pred)
    return pred

In [13]:
pred = fit_maxent_classifier(x_train, y_train, x_val)

test_pred = fit_maxent_classifier(x_train, y_train, x_test)

predict_1819 = fit_maxent_classifier(x_train, y_train, fixtures_1819)
print(predict_1819)
print(len(predict_1819))
for i in range(0, len(predict_1819)):
    print(i+2, ": ", predict_1819[i])

# print(y_val)

correct = 0
total = len(y_val)
for i in range(0, len(y_val)):
    if (y_val[i]==pred[i]):
        correct+=1
        
print(correct)
print(total)
print(correct/total)


correct = 0
total = len(y_test)
for i in range(0, len(y_test)):
    if (y_test[i]==test_pred[i]):
        correct+=1
        
print(correct)
print(total)
print(correct/total)

NameError: name 'fit_maxent_classifier' is not defined

In [ ]:
def keras_nn_classifier(learning_rate):
    # create the model
    model = Sequential()
    
    # add the layers
    model.add(Dense(1000, input_dim = num_features, kernel_regularizer = regularizers.l2(0.1), activation = 'sigmoid')) # hidden layer
    # model.add(Dropout(0.5))
    # model.add(Dense(750, kernel_regularizer = regularizers.l2(0.01), activation = 'sigmoid')) # second hidden layer
    #model.add(Dropout(0.3))
    #model.add(Dense(300, activation = 'sigmoid')) # second hidden layer
    # model.add(Dense(100, activation = 'sigmoid')) # second hidden layer
    #model.add(Dense(25, activation = 'sigmoid')) # second hidden layer

    model.add(Dense(num_classes, activation = 'softmax')) # output layer
        
    # optimizers
    sgd = keras.optimizers.SGD(lr = learning_rate) # model doesn't learn 
    adam = keras.optimizers.Adam(lr = learning_rate)   
    rmsprop = keras.optimizers.RMSprop(lr = learning_rate) 
    adagrad = keras.optimizers.Adagrad(lr = learning_rate) # bad
    adadelta = keras.optimizers.Adadelta(lr = learning_rate) # bad
    adamax = keras.optimizers.Adamax(lr = learning_rate) # not terrible, like 0.55
    nadam = keras.optimizers.Nadam(lr = learning_rate) # pretty good, 0.58
    
    # compile model
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer = adam, metrics = ['accuracy'])
    print(model.summary())
    return model

In [ ]:

#batch_size = 450
# tune more hyperparameters
epochs = 100
learning_rate = 1e-2
verb = 2
model = keras_nn_classifier(learning_rate)
# convert data into numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_val.shape)
print(y_val.shape)
print(x_train.shape)
print(y_train.shape)
history = model.fit(x_train, y_train, 
                   epochs = epochs, validation_data = (x_val, y_val),
                   verbose = verb, shuffle = True)
pre = model.predict(x_test)
print(pre)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
sum = 0.0
for val in history.history['acc']:
    sum+=val
avg = sum/epochs
print("Average Training Accuracy: ", avg)


sum = 0.0
for val in history.history['val_acc']:
    sum+=val
avg = sum/epochs
print("Average Validation Accuracy: ", avg)

print("Last Training Accuracy: ", history.history['acc'][epochs-1])
print("Last Validation Accuracy: ", history.history['val_acc'][epochs-1])


sum = 0.0
for i in range (epochs-10, epochs):
    sum+= history.history['val_acc'][i]
avg = sum / 10.0
print("Average of last 10 validation Accuracies: ", avg)

In [ ]:
clf = RandomForestClassifier(n_jobs = 10, n_estimators = 100, max_features = num_features, max_depth = 15, 
                             min_samples_split = 100, min_samples_leaf = 30, min_weight_fraction_leaf = 0., 
                            max_leaf_nodes = 50, min_impurity_decrease = 0.001, bootstrap = True, 
                            oob_score = False, verbose = 0, criterion = "gini")
clf.fit(x_train, y_train)
predicted = clf.predict(x_val)
correct = 0
for i in range(0, len(predicted)):
    if (y_val[i][0] == predicted[i][0]):
        if (y_val[i][1] == predicted[i][1]):
            #if (y_val[i][2] == predicted[i][2]):
                correct+=1
print(correct / len(predicted))
print(correct)


In [ ]:
def predictTestData():
    print(model.predict(x_test))

In [ ]:
predictTestData()